In this notebook, we compute weights for the Inosines dsRNA simulation which defines 3 different ensembles: MD; MD+NMR; MD+NMR*SAXS. 

In [3]:
#files bias$k ("bias"+str(k)) are computed by reprocessing the demuxed concateneted trajectory for each replica.
#SKIP this cell and load Bias array in the next cell

import plumed
import numpy as np

Bias=[]
for k in range(8):
    bias=plumed.read_as_pandas("bias"+str(k))
    MEW=plumed.read_as_pandas("MEbias"+str(k))
    bias["sum"]=bias["metad5.bias"]
    for i in ((6,7,8,9,10,11,12,13,14,15,16,25,26,27,28,29,30,31,32,33,34,35,36)):
        bias["sum"]+=bias["metad"+str(i)+".bias"]
    bias["sum"]+=(MEW["JCI9"]+MEW["JCI10"]+MEW["JCU11"]+MEW["JCI12"]+MEW["JCU23"]+MEW["JCU29"]+MEW["JCI30"]+MEW["JCU31"]+MEW["JCU32"])
    Bias.append(bias["sum"])
Bias=np.array(Bias)
Bias=np.transpose(Bias)
for j in range(8):
    Bias[:,j]-=np.min(Bias[:,j])
Bias 

ModuleNotFoundError: No module named 'plumed'

In [12]:
from importlib.machinery import SourceFileLoader
#bl = SourceFileLoader("bussilab","/net/sbp/sbpstore1/vpiompon/py-bussilab/bussilab/__init__.py").load_module()
bl = SourceFileLoader("bussilab","/u/area/vpiomponi/scratch/py-bussilab/bussilab/__init__.py").load_module()


import numpy as np
Bias=np.load('Bias.npy')


a =bl.wham.wham(bias=Bias, T=2.494, maxiter=10000)

Back-calculate J-couplings from the trajecotry using Barnaba python package. Davies parameters for the J_H1H2 must be implemented directly in the barnaba code (Condon parameters are the default values).

The demuxed concatenated trajecotry was divided in 5 parts here in orfer to allow loading

Skip to next cell to load directly J_H1H2 computed from the trajectory.

In [9]:
from importlib.machinery import SourceFileLoader
bb = SourceFileLoader("barnbaba","/net/sbp/sbpstore1/vpiompon/barnaba/barnaba/__init__.py").load_module()
bb



top = "rna_ions.pdb"
list=[]
 
list_couplings=[]
for k in (0,1,2,3,4,5,6,7):
# define trajectory and topology files
    traj = "traj_"+str(k)+"_a.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_b.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_c.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_d.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings[:-1])
    traj = "traj_"+str(k)+"_e.xtc"
    couplings,rr = bb.jcouplings(traj,topology=top, residues=['I_9_0','I_10_0','U_11_0','I_12_0','U_23_0','U_29_0','I_30_0','U_31_0','U_32_0'],couplings=['H1H2'])
    list_couplings.append(couplings)
        
        
    
list_couplings=np.concatenate((list_couplings[0],list_couplings[1]))
np.save('Davies_couplings.npy', list_couplings)

FileNotFoundError: [Errno 2] No such file or directory: '/net/sbp/sbpstore1/vpiompon/barnaba/barnaba/__init__.py'

In [13]:
list_couplings=np.load('Davies_couplings.npy')
Exp=[0.84,4.624,5.52,2.71,0.834,1.568,5.752,5.056,5.136,1.95**2]

MD+NMR ensemble

In [14]:
b=bl.maxent.maxent(traj=np.array(list_couplings).reshape(-1,9), reference=Exp[0:9], logW=a.logW)

In [17]:
nw=np.exp(-np.sum(((list_couplings).reshape(-1,9)*b.lambdas),axis=1))
np.save("MD_ensemble/wei.npy",np.exp(a.logW))
np.save("MD+NMR_ensemble/wei.npy",np.exp(a.logW)*nw)

MD+NMR+SAXS ensemble

In [21]:
rG=np.loadtxt("rg_file.xtx")[:,1]
np.shape(np.array(rG))

(2800008,)

In [23]:
traj_values=np.concatenate((np.array(list_couplings).reshape(-1,9),np.square(np.array(rG))[:,np.newaxis]),axis=1)

In [24]:
from bussilab import maxent

b=maxent.maxent(traj=traj_values, reference=Exp, logW=a.logW)
nw=np.exp(-np.sum(((traj_values).reshape(-1,10)*b.lambdas),axis=1))


In [25]:
np.save("MD+NMR+SAXS_ensemble/wei.npy",np.exp(a.logW)*nw)